In [19]:
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import export_graphviz
import sqlite3

In [20]:
conn = sqlite3.connect('../HomelessData.db')
c = conn.cursor()

In [21]:
# Loading data
df_2018 = pd.read_sql("SELECT * FROM CocData WHERE year = 2018", con = conn)

df_2018.head()

,Coc,Total,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Year,CocYear,Total:1,Unsheltered,Sheltered
0,AK-500,1028,749,279,0,125,604,71,2018,AK-500 2018,1094,94,1000
1,AK-501,912,649,263,0,90,344,0,2018,AK-501 2018,922,215,707
2,AL-500,911,569,308,34,191,1768,0,2018,AL-500 2018,901,219,682
3,AL-501,435,298,137,0,60,203,0,2018,AL-501 2018,551,233,318
4,AL-502,210,102,108,0,0,50,0,2018,AL-502 2018,256,22,234


In [22]:
df_2018=df_2018.drop(['Total:1', 'Sheltered', 'Total', 'CocYear', 'Coc', 'Year'], axis=1)

In [24]:
df_2018.head()

,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Unsheltered
0,749,279,0,125,604,71,94
1,649,263,0,90,344,0,215
2,569,308,34,191,1768,0,219
3,298,137,0,60,203,0,233
4,102,108,0,0,50,0,22


In [ ]:
# Define the features set.
X = df_NAME.copy()
X = X.drop("________", axis=1)
X.head()

In [ ]:
# Define the target set.
y = df_NAME["________"].ravel()
y[:5]

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
regr = RandomForestRegressor(max_depth=3, random_state=0)

In [ ]:
reg_model = regr.fit(X, y)

In [ ]:
importances = reg_model.feature_importances_
importances

In [ ]:
# We can sort the features by their importance.
sorted(zip(reg_model.feature_importances_, X.columns), reverse=True)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
features = sorted(zip(X.columns, reg_model.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(10,20)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

In [ ]:
estimator = reg_model.estimators_[5]

In [ ]:
from sklearn import tree
tree.plot_tree(reg_model.estimators_[0])